## Spark Session Setup

In [2]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("Advanced Spark") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()

### Loading Data

In [3]:
listings = spark.read.csv("../data/listings.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",", 
    quote='"',
    escape='"', 
    multiLine=True,
    mode="PERMISSIVE" 
)
listings.printSchema()

root
 |-- id: long (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: timestamp (nullable = true)
 |-- source: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: timestamp (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_

In [4]:
reviews = spark.read.csv("../data/reviews.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",",
    quote='"',
    escape='"',
    multiLine=True,
    mode="PERMISSIVE"
)
reviews.printSchema()

root
 |-- listing_id: long (nullable = true)
 |-- id: long (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- reviewer_id: integer (nullable = true)
 |-- reviewer_name: string (nullable = true)
 |-- comments: string (nullable = true)



In [5]:
# 1. For each listing compute string category depending on its price, and add it as a new column.
# A category is defined in the following way:
#
# * price < 50 -> "Budget"
# * 50 <= price < 150 -> "Mid-range"
# * price >= 150 -> "Luxury"
# 
# Only include listings where the price is not null.
# Count the number of listings in each category

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import regexp_replace

listings = listings.withColumn('price_numeric', regexp_replace('price', '[$,]', '').cast('float'))

def categorize_price(price):
    if price is None:
        return 'Unknown'
    elif price < 50:
        return 'Budget'
    elif 50 <= price < 150:
        return 'Mid-range'
    elif price >= 150:
        return 'Luxury'
    else:
        return 'Unknown'


categorize_price_udf = udf(categorize_price, StringType())


In [6]:
listings_with_category = listings \
  .filter(listings.price_numeric.isNotNull()) \
  .withColumn(
    'price_category',
    categorize_price_udf(listings.price_numeric)
  ) \
  .groupBy('price_category') \
  .count() \
  .orderBy('count', ascending=False) 




In [14]:
# 2. In this task you will need to compute a santiment score per review, and then an average sentiment score per listing.
# A santiment score indicates how "positive" or "negative" a review is. The higher the score the more positive it is, and vice-versa.
#
# To compute a sentiment score per review compute the number of positive words in a review and subtract the number of negative
# words in the same review (the list of words is already provided)
#
# To complete this task, compute a DataFrame that contains the following fields:
# * name - the name of a listing
# * average_sentiment - average sentiment of reviews computed using the algorithm described above

from pyspark.sql.functions import avg
from pyspark.sql.types import FloatType

positive_words = ['good', 'great', 'excellent', 'amazing', 'fantastic', 'wonderful', 'pleasant', 'lovely', 'nice', 'enjoyed']
negative_words = ['bad', 'terrible', 'awful', 'horrible', 'disappointing', 'poor', 'hate', 'unpleasant', 'dirty', 'noisy']

def sentiment_score(comment):
    if comment is None:
        return 0.0
    comment_lower = comment.lower()
    score = 0

    for word in positive_words:
        if word in comment_lower:
            score += 1

    for word in negative_words:
        if word in comment_lower:
            score -= 1
    return float(score)

sentiment_score_udf = udf(sentiment_score, FloatType())



In [15]:
test_comment = "This place was great and wonderful!"
result = sentiment_score(test_comment)
print(f"Result: {result}")


Result: 2.0


In [16]:
reviews.printSchema()

root
 |-- listing_id: long (nullable = true)
 |-- id: long (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- reviewer_id: integer (nullable = true)
 |-- reviewer_name: string (nullable = true)
 |-- comments: string (nullable = true)



In [17]:
reviews_with_sentiment = reviews \
  .withColumn(
    'sentiment_score',
    sentiment_score_udf(reviews.comments)
  )

listings \
   .join(reviews_with_sentiment, listings.id == reviews.listing_id, 'inner') \
   .groupBy('listing_id', 'name') \
   .agg(
      avg('sentiment_score').alias('average_sentiment')
   ) \
   .orderBy('average_sentiment', ascending=False) \
   .select('listing_id', 'name', 'average_sentiment') \
   .show(truncate=False)

Traceback (most recent call last):
  File "c:\Users\Shah\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Shah\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "c:\Users\Shah\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Shah\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 692, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Shah\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 565, in _function

PicklingError: Could not serialize object: IndexError: tuple index out of range

In [19]:
from pyspark.sql.functions import when, col, lower, length

positive_words = ['good', 'great', 'excellent', 'amazing', 'fantastic', 'wonderful', 'pleasant', 'lovely', 'nice', 'enjoyed']
negative_words = ['bad', 'terrible', 'awful', 'horrible', 'disappointing', 'poor', 'hate', 'unpleasant', 'dirty', 'noisy']

# Avoid UDF completely - use built-in functions
# Create regex patterns for positive and negative words
positive_pattern = '|'.join([f'\\b{word}\\b' for word in positive_words])
negative_pattern = '|'.join([f'\\b{word}\\b' for word in negative_words])

reviews_with_sentiment = reviews.limit(20) \
  .withColumn(
    'sentiment_score',
    when(col('comments').isNull(), 0.0)
    .when(lower(col('comments')).rlike(positive_pattern), 1.0)
    .when(lower(col('comments')).rlike(negative_pattern), -1.0)
    .otherwise(0.0)
  )

reviews_with_sentiment.show()

+----------+---------+-------------------+-----------+-------------+--------------------+---------------+
|listing_id|       id|               date|reviewer_id|reviewer_name|            comments|sentiment_score|
+----------+---------+-------------------+-----------+-------------+--------------------+---------------+
|     13913|    80770|2010-08-18 00:00:00|     177109|      Michael|My girlfriend and...|            1.0|
|     13913|   367568|2011-07-11 00:00:00|   19835707|      Mathias|Alina was a reall...|            1.0|
|     13913|   529579|2011-09-13 00:00:00|    1110304|      Kristin|Alina is an amazi...|            1.0|
|     13913|   595481|2011-10-03 00:00:00|    1216358|      Camilla|Alina's place is ...|            1.0|
|     13913|   612947|2011-10-09 00:00:00|     490840|        Jorik|Nice location in ...|            1.0|
|     13913|  4847959|2013-05-28 00:00:00|    6405442|         Vera|I'm very happy to...|            1.0|
|     13913|  8142329|2013-10-17 00:00:00|    